In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

# NOTE: Revisit training loops and adjust for tf.GradientTape if applicable.


In [2]:
df = pd.read_csv('fashion.csv')
df.head()

,date,Cami Dresses,Shirts,Tote Bags,Sneakers,Crop Tops,Polos,Cross Body Bags,Casual Jackets,Swimwear,...,Heels,T-Shirts,Activewear Tops & T-Shirts,Watches & Timepieces,Wallets & Card Holders,Bodycon Dresses,Beauty Tools & Accessories,Skinny Jeans,Beauty Eyes,Beauty Face
0,2017-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-08-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-08-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-08-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
df = df.iloc[:,1:]
df_copy = df.copy()

In [12]:
num_layers = 1
learning_rate = 0.01
size_layer = 128
timestamp = 5
epoch = 500
dropout_rate = 0.7
future_weeks = 30

In [21]:
class Model(tf.keras.Model):
    def __init__(self, learning_rate, num_layers, size, size_layer, forget_bias=0.8):
        super(Model, self).__init__()
        
        self.num_layers = num_layers
        self.size_layer = size_layer
        
        # Define LSTM layers
        self.lstm_layers = [
            tf.keras.layers.LSTM(
                units=size_layer,
                return_sequences=True if i < num_layers - 1 else False,
                return_state=True,
                dropout=1 - forget_bias,
                recurrent_dropout=1 - forget_bias
            )
            for i in range(num_layers)
        ]
        
        # Fully connected (dense) layer applied to each time step
        self.dense = tf.keras.layers.Dense(
            units=size,
            kernel_initializer=tf.keras.initializers.GlorotUniform()
        )
        
        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
        # Loss function
        self.loss_fn = tf.keras.losses.MeanSquaredError()

    def call(self, inputs, initial_states):
        """
        Forward pass through the model.
        Args:
            inputs: Input tensor of shape (batch_size, time_steps, size).
            initial_states: List of initial hidden states for LSTM layers.
        Returns:
            logits: Output predictions from the dense layer.
            new_states: Updated hidden states from LSTM layers.
        """
        x = inputs
        states = initial_states
        new_states = []
        
        for i, lstm in enumerate(self.lstm_layers):
            x, h_state, c_state = lstm(x, initial_state=states[i])
            new_states.append([h_state, c_state])
        
        logits = self.dense(x)  # Dense layer applied to each time step
        return logits, new_states

    def train_step(self, x, y, initial_states):
        """
        Perform one training step.
        Args:
            x: Input tensor.
            y: Target tensor.
            initial_states: List of initial hidden states for LSTM layers.
        Returns:
            loss: Computed loss for the batch.
            new_states: Updated hidden states.
        """
        with tf.GradientTape() as tape:
            logits, new_states = self.call(x, initial_states)
            
            # Ensure shapes match by reshaping target if necessary
            y = tf.reshape(y, logits.shape)
            
            loss = self.loss_fn(y, logits)
        
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        return loss, new_states


In [24]:
# Graph resets not needed in TF2.x
modelnn = Model(learning_rate, num_layers, df.shape[1], size_layer, dropout_rate)
# Initialize model
# modelnn = Model(learning_rate, num_layers, size, size_layer, forget_bias)

# Dummy data
X = tf.random.normal((batch_size, sequence_length, size))
Y = tf.random.normal((batch_size, df.shape[1]))

# Initial hidden states for each LSTM layer
initial_states = [[
    tf.zeros((batch_size, size_layer)),
    tf.zeros((batch_size, size_layer))
] for _ in range(num_layers)]



In [25]:
# Assuming the `Model` class from earlier and `df` are defined
epoch = 1000
timestamp = 10
batch_size = 1

for i in range(epoch):
    # Initialize the hidden states for the LSTM layers
    init_value = [
        [tf.zeros((batch_size, size_layer)), tf.zeros((batch_size, size_layer))]
        for _ in range(num_layers)
    ]
    
    total_loss = 0.0
    for k in range(0, (df.shape[0] // timestamp) * timestamp, timestamp):
        # Prepare the batch data
        batch_x = np.expand_dims(df.iloc[k: k + timestamp].values, axis=0).astype(np.float32)
        batch_y = df.iloc[k + 1: k + timestamp + 1].values.astype(np.float32)

        # Convert to tensors
        batch_x = tf.convert_to_tensor(batch_x)
        batch_y = tf.convert_to_tensor(batch_y)

        # Perform a training step
        loss, updated_states = modelnn.train_step(batch_x, batch_y, init_value)

        # Update the initial states with the updated states
        init_value = updated_states
        total_loss += loss.numpy()

    # Compute average loss for the epoch
    total_loss /= (df.shape[0] // timestamp)

    # Log progress every 100 epochs

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 450 values, but the requested shape has 45 [Op:Reshape]

In [ ]:
output_predict = np.zeros((df.shape[0] + future_weeks, df.shape[1]))
output_predict[0, :] = df.iloc[0] 
upper_b = (df.shape[0] // timestamp) * timestamp
init_value = np.zeros((1, num_layers * 2 * size_layer))
for k in range(0, (df.shape[0] // timestamp) * timestamp, timestamp):
    out_logits, last_state = # Replace sess.run with direct calls or @tf.function in TF2.x([tf.nn.sigmoid(modelnn.logits), modelnn.last_state], 
                                      feed_dict = {modelnn.X:np.expand_dims(df.iloc[k: k + timestamp], axis = 0),
                                     modelnn.hidden_layer: init_value})
    init_value = last_state
    output_predict[k + 1: k + timestamp + 1] = out_logits
    
out_logits, last_state = # Replace sess.run with direct calls or @tf.function in TF2.x([tf.nn.sigmoid(modelnn.logits), modelnn.last_state], 
                                  feed_dict = {modelnn.X:np.expand_dims(df.iloc[upper_b:], axis = 0),
                                     modelnn.hidden_layer: init_value})
init_value = last_state
output_predict[upper_b + 1: df.shape[0] + 1] = out_logits
df.loc[df.shape[0]] = out_logits[-1]
date_ori.append(date_ori[-1]+timedelta(days=3))

# NOTE: Revisit training loops and adjust for tf.GradientTape if applicable.


In [ ]:
for i in range(future_weeks - 1):
    out_logits, last_state = # Replace sess.run with direct calls or @tf.function in TF2.x([tf.nn.sigmoid(modelnn.logits), modelnn.last_state], feed_dict = 
                                      {modelnn.X:np.expand_dims(df.iloc[-timestamp:], axis = 0),
                                     modelnn.hidden_layer: init_value})
    init_value = last_state
    output_predict[df.shape[0], :] = out_logits[-1, :]
    df.loc[df.shape[0]] = out_logits[-1, :]
    date_ori.append(date_ori[-1]+timedelta(days=3))

# NOTE: Revisit training loops and adjust for tf.GradientTape if applicable.


In [ ]:
date_ori=pd.Series(date_ori).dt.strftime(date_format='%Y-%m-%d').tolist()

In [ ]:
index = (-np.round(df.values).sum(axis=0)).argsort()[4:10]
index

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
fig = plt.figure(figsize = (15,15))
for no, i in enumerate(index):
    plt.subplot(6,1,no+1)
    label = list(df)[i]
    plt.plot(np.around(df.iloc[:,i]),label='predicted ' + label,alpha=0.7)
    plt.plot(np.around(df_copy.iloc[:,i]),label='real ' + label,alpha=0.7)
    plt.legend()
    x_range_future = np.arange(df.shape[0])
    plt.xticks(x_range_future[::20], date_ori[::20])
plt.show()
plt.show()

In [ ]:
def df_shift(df,lag=0,rejected_columns = []):
    df = df.copy()
    if not lag:
        return df
    cols ={}
    for i in range(1,lag+1):
        for x in list(df.columns):
            if x not in rejected_columns:
                if not x in cols:
                    cols[x] = ['{}_{}'.format(x, i)]
                else:
                    cols[x].append('{}_{}'.format(x, i))
    for k,v in cols.items():
        columns = v
        dfn = pd.DataFrame(data=None, columns=columns, index=df.index)    
        i = 1
        for c in columns:
            dfn[c] = df[k].shift(periods=i)
            i+=1
        df = pd.concat([df, dfn], axis=1, join_axes=[df.index])
    return df

In [ ]:
df_new = df_shift(df, 2)
df_new.shape

In [ ]:
df_new.head()

In [ ]:
df_new = df_new.dropna()

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(15,10))
plt.title('2 days correlation', y=1.05, size=16)

mask = np.zeros_like(df_new.corr())
mask[np.triu_indices_from(mask)] = True

sns.heatmap(df_new.corr(), mask=mask, linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=False)
plt.show()